# Load Libraries and Functions

In [7]:
# System, Data, Time, and Spec Libraries
import os
import pandas as pd
import time
import random
from datetime import datetime, timedelta
import numpy as np 
from line_profiler import LineProfiler  # Code peformance
profiler = LineProfiler()
import warnings
warnings.filterwarnings('ignore')
import csv
#pd.set_option('display.max_colwidth', None)
import multiprocess as mp
num_cores = mp.cpu_count()

# Data Visualization Libraries
import matplotlib.pyplot as plt
from tabulate import tabulate
import plotly.express as px
import seaborn as sns
#from pandas.io.json import json_normalize  # Older version
from pandas import json_normalize  # Newer version
from pandas.plotting import parallel_coordinates


# Natural Language Processing Libraries
import json
import requests
import xml.etree.ElementTree as ET
import nltk
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import words
import string
import nltk
from collections import OrderedDict
from nltk.tokenize import word_tokenize
from pandas import json_normalize
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from nltk.util import ngrams
from nltk.corpus import stopwords
from string import punctuation
punctuation = set(punctuation)
punctuation.update({'_', '-','‘'})
english_words = set(words.words())
from fuzzywuzzy import process
#nltk.download('words')
#nltk.download('punkt')
#nltk.download('stopwords')

# SQL Interface Libraries
import pymysql as mysql
import mysql.connector
import pyodbc
import sqlite3
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy import event
from string import punctuation

# Machine Learning Libraries
import scipy.stats as stats
from scipy.stats import zscore
from scipy.stats import yeojohnson
import statsmodels.api as sm
from sklearn.preprocessing import OneHotEncoder, PowerTransformer
from sklearn.impute import KNNImputer
from sklearn.feature_selection import VarianceThreshold
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import confusion_matrix, accuracy_score, ConfusionMatrixDisplay,roc_auc_score, roc_curve 
from sklearn.metrics import classification_report, mean_squared_error, f1_score
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline, Pipeline
from dmba import classificationSummary, AIC_score, BIC_score, plotDecisionTree,gainsChart
from scikitplot.metrics import plot_lift_curve, plot_cumulative_gain
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.ensemble import GradientBoostingClassifier
from imblearn.metrics import specificity_score, sensitivity_score
from imblearn.under_sampling import RandomUnderSampler
from scikitplot.metrics import plot_lift_curve
from sklearn.impute import SimpleImputer
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn import preprocessing
from sklearn import tree
from sklearn.model_selection import cross_val_score
from patsy import dmatrices
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import KFold
from dmba import stepwise_selection, classificationSummary, backward_elimination
from sklearn.decomposition import PCA
from sklearn.utils.random import sample_without_replacement
from sklearn.metrics import precision_score, recall_score

# Importing Custom Functions
import nbimporter
from Functions import nan_info

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import numpy as np

# Test Join Outside of Database

In [68]:
# df1 = pd.read_csv('DataLibrary/events_table2023.csv')
# df2 = pd.read_csv('DataLibrary/patient_drugs_table2023.csv')
# df3 = pd.read_csv('DataLibrary/reactions_table2023.csv')

# # Merge df1 and df2
# merged_df1_df2 = pd.merge(df1, df2, on='event_id', how='inner')

# # Merge the result with df3
# final_merged_df = pd.merge(merged_df1_df2, df3, on='event_id', how='inner')

# # Display the final merged DataFrame
# print(final_merged_df)

In [6]:
merged_df1_df2.shape

(133423, 20)

### SQL Password

In [3]:
PASSWORD  = 'PASSWORD' 

## Investigate number of records in each database table

In [69]:
connection = mysql.connector.connect(
    host="localhost", user="root", password=PASSWORD, database="pharma_db"
)

cursor = connection.cursor()
sql_query = """SELECT COUNT(*) FROM patient_reactions"""


cursor.execute(sql_query)
result = cursor.fetchall()
column_names = [i[0] for i in cursor.description]


result_query_df = pd.DataFrame(result, columns=column_names)

cursor.close()
connection.close()

result_query_df

,COUNT(*)
0,112551


# Get Dataset from SQL Database

In [70]:
# Connect to the MySQL server
connection = mysql.connector.connect(
    host="localhost", user="root", password=PASSWORD, database="pharma_db"
)

# Create a cursor object
cursor = connection.cursor()

In [15]:
# # Master Query from Data - goal

# master_query = """SELECT 
#                 a.serious_outcome,
#                 a.expedited,
#                 a.age,
#                 a.sex,
#                 a.year
#                 a.weight
#                 r.outcome,
#                 p.unit_price,
#                 p.generic_brand,
#                 l.ingredients,
#                 l.rxcui,
#                 l.set_id,
#                 d.manu_num,
#                 d.unii
#             FROM adverse_events a 
#                 INNER JOIN patients_reactions r ON a.event_id = r.event_id 
#                 INNER JOIN patients_drugs d ON r.event_id = d.event_id 
#                 INNER JOIN prices p ON d.ndc11 = p.ndc11
#                 INNER JOIN lables l ON p.ndc11 = l.ndc11
#             ORDER BY a.event_id DESC"""  # Still need to test and figure out

# Read in data from csv OR

In [8]:
result_query_df = pd.read_csv('DataLibrary/result_query_df.csv')

# option 2- Read in data from SQL

In [76]:
start_time = time.time()

connection = mysql.connector.connect(
    host="localhost", user="root", password=PASSWORD, database="pharma_db"
)

cursor = connection.cursor()
sql_query = """SELECT 
d.med_product,
d.event_id,
d.manu_num,
a.serious_outcome,
a.expedited,
a.age,
a.sex,
a.year,
a.weight,
r.outcome
FROM adverse_events a 

INNER JOIN patient_reactions r ON a.event_id = r.event_id
INNER JOIN patient_drugs d ON a.event_id = d.event_id
ORDER BY a.event_id DESC
"""
cursor.execute(sql_query)
result = cursor.fetchall()
column_names = [i[0] for i in cursor.description]
result_query_df = pd.DataFrame(result, columns=column_names)

cursor.close()
connection.close()


end_time = time.time()
execution_time = end_time - start_time
print(execution_time)

result_query_df.head(10)


47.15399479866028


,med_product,event_id,manu_num,serious_outcome,expedited,age,sex,year,weight,outcome
0,['stelara'],24685,1,2,1,64,1,2024,72,Fatal
1,['stelara'],24685,1,2,1,64,1,2024,72,Fatal
2,['remicade'],24685,1,2,1,64,1,2024,72,Fatal
3,['cyclosporine'],24685,16,2,1,64,1,2024,72,Fatal
4,['efalizumab'],24685,0,2,1,64,1,2024,72,Fatal
5,['stelara'],24685,1,2,1,64,1,2024,72,Fatal
6,['stelara'],24685,1,2,1,64,1,2024,72,Fatal
7,['remicade'],24685,1,2,1,64,1,2024,72,Fatal
8,['cyclosporine'],24685,16,2,1,64,1,2024,72,Fatal
9,['efalizumab'],24685,0,2,1,64,1,2024,72,Fatal


In [26]:
result_query_df.shape

(99193, 11)

In [27]:
result_query_df.to_csv('DataLibrary/result_query_df.csv')

In [9]:
result_query_df = result_query_df.drop_duplicates()
result_query_df.shape

(99193, 12)

# Make dataset smaller for testing

In [10]:
result_query_df = result_query_df.head(5000)

In [11]:
master_query_df = result_query_df

# Preparation for Data Split

In [12]:
master_query_df.head()

,Unnamed: 0.1,Unnamed: 0,med_product,event_id,manu_num,serious_outcome,expedited,age,sex,year,weight,outcome
0,0,0,['stelara'],24685,1,2,1,64,1,2024,72,Fatal
1,1,2,['remicade'],24685,1,2,1,64,1,2024,72,Fatal
2,2,3,['cyclosporine'],24685,16,2,1,64,1,2024,72,Fatal
3,3,4,['efalizumab'],24685,0,2,1,64,1,2024,72,Fatal
4,4,10,['lansoprazole'],24684,56,2,1,75,2,2024,63,Fatal


In [13]:
nan_info(master_query_df)

,column_name,null_count,null_proportion
0,Unnamed: 0.1,0,0.0
1,Unnamed: 0,0,0.0
2,med_product,0,0.0
3,event_id,0,0.0
4,manu_num,0,0.0
5,serious_outcome,0,0.0
6,expedited,0,0.0
7,age,0,0.0
8,sex,0,0.0
9,year,0,0.0


### Update Datatypes

In [14]:
master_query_df.info()
result_query_df['serious_outcome'] = result_query_df['serious_outcome'].astype('category')

result_query_df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 0 to 4999
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Unnamed: 0.1     5000 non-null   int64 
 1   Unnamed: 0       5000 non-null   int64 
 2   med_product      5000 non-null   object
 3   event_id         5000 non-null   int64 
 4   manu_num         5000 non-null   int64 
 5   serious_outcome  5000 non-null   int64 
 6   expedited        5000 non-null   int64 
 7   age              5000 non-null   int64 
 8   sex              5000 non-null   int64 
 9   year             5000 non-null   int64 
 10  weight           5000 non-null   int64 
 11  outcome          5000 non-null   object
dtypes: int64(10), object(2)
memory usage: 507.8+ KB


,Unnamed: 0.1,Unnamed: 0,med_product,event_id,manu_num,serious_outcome,expedited,age,sex,year,weight,outcome
0,0,0,['stelara'],24685,1,2,1,64,1,2024,72,Fatal
1,1,2,['remicade'],24685,1,2,1,64,1,2024,72,Fatal
2,2,3,['cyclosporine'],24685,16,2,1,64,1,2024,72,Fatal
3,3,4,['efalizumab'],24685,0,2,1,64,1,2024,72,Fatal
4,4,10,['lansoprazole'],24684,56,2,1,75,2,2024,63,Fatal


### Define numerical, text, and categorical variables

In [15]:
cats = ['sex', 'expedited']
nums = ['weight', 'year', 'age'] 
#texts = ['']
#all_vars = cats+nums+texts
all_vars = cats+nums

In [16]:
master_query_df.head()

,Unnamed: 0.1,Unnamed: 0,med_product,event_id,manu_num,serious_outcome,expedited,age,sex,year,weight,outcome
0,0,0,['stelara'],24685,1,2,1,64,1,2024,72,Fatal
1,1,2,['remicade'],24685,1,2,1,64,1,2024,72,Fatal
2,2,3,['cyclosporine'],24685,16,2,1,64,1,2024,72,Fatal
3,3,4,['efalizumab'],24685,0,2,1,64,1,2024,72,Fatal
4,4,10,['lansoprazole'],24684,56,2,1,75,2,2024,63,Fatal


In [17]:
master_query_df.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'med_product', 'event_id', 'manu_num',
       'serious_outcome', 'expedited', 'age', 'sex', 'year', 'weight',
       'outcome'],
      dtype='object')

In [18]:
all_vars

['sex', 'expedited', 'weight', 'year', 'age']

### Create Transformation Pipeline

In [19]:
# # Create a categorical processing pipeline that uses one-hot encoding
# # Dropping binary columns and drop first of each level** NEED TO ADD**
# cat_pipe = Pipeline([('encoder', OneHotEncoder(drop='if_binary'))])

# # Create a numerical processing pipeline that uses skewness correction/center/scale.
# num_pipe = Pipeline([('skew_standardize', PowerTransformer())])

# # Create a text token processing step to vectorize tokens
# #text_pipe = Pipeline([('vector', tf_idf function())])

# # Combine pipeline steps
# all_pipe = ColumnTransformer(
#     transformers=[
#         ('cat', cat_pipe, cats),
#         ('num', num_pipe, nums)
#         # ('text', text_pipe, texts)  # Apply text pipeline if you have text columns
#     ],
#     verbose_feature_names_out=False
# )
# # Verify steps
# all_pipe

# Split Data into Training/Validation/Testing

In [20]:
# Split the data into features (X) and the target variable (y).
X = master_query_df[all_vars]

#Define outcome variable
y = master_query_df[['serious_outcome']]  # Need to Decide 5 Level or 3 Level

#Split data
X_train, X_rest, y_train, y_rest = train_test_split(X, y, train_size=0.8, random_state = 2)
X_val, X_test, y_val, y_test = train_test_split(X_rest, y_rest, test_size=0.5, random_state = 2)

In [21]:
X_val.head()

,sex,expedited,weight,year,age
213,2,2,55,2024,83
2247,2,1,97,2024,45
1460,2,1,60,2024,64
892,1,1,60,2024,73
273,2,1,68,2024,43


In [22]:
X_test.shape, y_test.shape

((500, 5), (500, 1))

In [23]:
X_val.shape, y_val.shape

((500, 5), (500, 1))

## Apply Pipeline to Splits Separately to Prevent Data Leakage

In [24]:
X_train.head()

,sex,expedited,weight,year,age
868,2,1,68,2024,17
936,2,1,72,2024,67
4034,2,1,74,2024,41
1923,2,1,65,2024,75
3330,2,1,15,2024,2


In [25]:
encoder = OneHotEncoder(drop='if_binary', sparse=False)

# Fit the encoder on X_train
encoder.fit(X[cats])

encoder = OneHotEncoder(drop='if_binary', sparse=False, handle_unknown='ignore')

# Fit the encoder on X_train
encoder.fit(X[cats])

# Function to transform data
def transform_data(data):
    # Apply one-hot encoding
    encoded_features = encoder.transform(data[cats])
    # Create a DataFrame for the encoded features
    encoded_df = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(cats))
    # Drop the original categorical columns and concatenate with the encoded features
    return pd.concat([data.drop(cats, axis=1).reset_index(drop=True), encoded_df.reset_index(drop=True)], axis=1)

# Apply the transformation to all datasets
X_train_encoded = transform_data(X_train)
X_val_encoded = transform_data(X_val)
X_test_encoded = transform_data(X_test)

# Print the results
print("X_train_encoded:")
X_train_encoded.head()
print("\nX_val_encoded:")
X_val_encoded.head()
print("\nX_test_encoded:")
X_test_encoded.head()

X_train_encoded:

X_val_encoded:

X_test_encoded:


,weight,year,age,sex_2,expedited_2
0,74,2024,84,1.0,0.0
1,102,2024,52,1.0,0.0
2,68,2024,17,1.0,0.0
3,50,2024,12,1.0,0.0
4,65,2024,67,1.0,0.0


In [26]:
# #Fit pipeline to resampled data
# X_train_fit = all_pipe.fit_transform(X_train)
# #Get feature names out from fit and create as new list

# def get_feature_names(column_transformer):
#     feature_names = []
#     for name, transformer, columns in column_transformer.transformers:
#         if hasattr(transformer, 'get_feature_names_out'):
#             feature_names.extend(transformer.get_feature_names_out())
#         else:
#             feature_names.extend(columns)  # Use original columns if transformer does not provide feature names
#     return feature_names

# # Extract feature names
# feature_names = get_feature_names(all_pipe)


# #X_train_cols = X_train_fit.get_feature_names_out().tolist()
# X_train_pipe = pd.DataFrame(all_pipe.fit_transform(X_train), columns = X_train_cols)

In [27]:
# #Apply pipeline to all of X
# X_test_fit = all_pipe.fit(X_test)
# #Get feature names out from fit and create as new list
# # X_test_cols = X_test_fit.get_feature_names_out().tolist()
# X_test_pipe = pd.DataFrame(all_pipe.fit_transform(X_test), columns = X_cols)

In [28]:
# #Apply pipeline to all of X
# X_val_fit = all_pipe.fit(X_val)
# #Get feature names out from fit and create as new list
# X_val_cols = X_val_fit.get_feature_names_out().tolist()
# X_val_pipe = pd.DataFrame(all_pipe.fit_transform(X_val), columns = X_cols)

## Undersample Training Data to Balance Outcome

In [29]:
# # Create a RandomUnderSampler instance with a specified random seed and sampling strategy
# rus = RandomUnderSampler(random_state = 1, sampling_strategy='majority')

# # Perform random under-sampling on the training dataset
# X_train_under, y_train_under = rus.fit_resample(X_train_pipe, y_train)

In [55]:
y_test.value_counts()

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.

serious_outcome
1                  406
2                   79
0                   15
dtype: int64

# Multiclass Classification Model Training

## White Box Models

### LOGISTIC REGRESSION

### Logistic Regression L1 Regularization

In [30]:
X_train_encoded.dtypes

weight           int64
year             int64
age              int64
sex_2          float64
expedited_2    float64
dtype: object

In [31]:
log_l1 = LogisticRegression(solver='saga', penalty='l1', random_state=1)
log_l1.fit(X_train_encoded, y_train)

# Intercept Log-Odds and Odds
print(log_l1.intercept_ , np.exp(log_l1.intercept_))

# Make predictions
y_pred = log_l1.predict(X_test_encoded)



# Make predictions on the encoded test data
y_pred = log_l1.predict(X_test_encoded)

# Compute the confusion matrix
cm = confusion_matrix(y_test, y_pred, labels=[0, 1, 2])

# Accuracy
logl1_acc = accuracy_score(y_test, y_pred)

# Precision, Recall (Sensitivity), and F1-Score for each class
logl1_prec = precision_score(y_test, y_pred, average=None, labels=[0, 1, 2])
logl1_rec = recall_score(y_test, y_pred, average=None, labels=[0, 1, 2])
logl1_f1 = f1_score(y_test, y_pred, average=None, labels=[0, 1, 2])

# Specificity calculation for each class
specificity = np.array([
    (np.sum(cm) - np.sum(cm[:, i]) - np.sum(cm[i, :]) + cm[i, i]) / (np.sum(cm) - np.sum(cm[:, i]))
    for i in range(cm.shape[0])
])

# Extract metrics for class 2 (index 2)
class_index = 2
logl1_prec_class2 = logl1_prec[class_index]
logl1_rec_class2 = logl1_rec[class_index]
logl1_f1_class2 = logl1_f1[class_index]
logl1_spec_class2 = specificity[class_index]

print(f"Accuracy: {logl1_acc:.2f}")
print(f"Class 2 Precision: {logl1_prec_class2:.2f}")
print(f"Class 2 Specificity: {logl1_spec_class2:.2f}")
print(f"Class 2 Recall: {logl1_rec_class2:.2f}")
print(f"Class 2 F1-Score: {logl1_f1_class2:.2f}")


[-5.82153479e-07  4.60484469e-07  1.21669011e-07] [0.99999942 1.00000046 1.00000012]
Accuracy: 0.81
Class 2 Precision: 0.00
Class 2 Specificity: 0.84
Class 2 Recall: 0.00
Class 2 F1-Score: 0.00


### Logistic Regression L2 Regularization

In [32]:
log_l2 = LogisticRegression(solver='saga', penalty='l2', random_state=1)
log_l2.fit(X_train_encoded, y_train)

# Intercept Log-Odds and Odds
print("Intercept (Log-Odds):", log_l2.intercept_)
print("Intercept (Odds):", np.exp(log_l2.intercept_))

# Make predictions on the encoded test data
y_pred = log_l2.predict(X_test_encoded)

# Compute the confusion matrix
cm = confusion_matrix(y_test, y_pred, labels=[0, 1, 2])

# Accuracy
logl2_acc = accuracy_score(y_test, y_pred)

# Precision, Recall (Sensitivity), and F1-Score for each class
logl2_prec = precision_score(y_test, y_pred, average=None, labels=[0, 1, 2])
logl2_rec = recall_score(y_test, y_pred, average=None, labels=[0, 1, 2])
logl2_f1 = f1_score(y_test, y_pred, average=None, labels=[0, 1, 2])

# Specificity calculation for each class
specificity = np.array([
    (np.sum(cm) - np.sum(cm[:, i]) - np.sum(cm[i, :]) + cm[i, i]) / (np.sum(cm) - np.sum(cm[:, i]))
    for i in range(cm.shape[0])
])

# Extract metrics for class 2 (index 2)
class_index = 2
logl2_prec_class2 = logl2_prec[class_index]
logl2_rec_class2 = logl2_rec[class_index]
logl2_f1_class2 = logl2_f1[class_index]
logl2_spec_class2 = specificity[class_index]

# Print metrics
print(f"Accuracy: {logl2_acc:.2f}")
print(f"Class 2 Precision: {logl2_prec_class2:.2f}")
print(f"Class 2 Specificity: {logl2_spec_class2:.2f}")
print(f"Class 2 Recall: {logl2_rec_class2:.2f}")
print(f"Class 2 F1-Score: {logl2_f1_class2:.2f}")

Intercept (Log-Odds): [-5.50949598e-07  4.44103496e-07  1.06846102e-07]
Intercept (Odds): [0.99999945 1.00000044 1.00000011]
Accuracy: 0.81
Class 2 Precision: 0.00
Class 2 Specificity: 0.84
Class 2 Recall: 0.00
Class 2 F1-Score: 0.00


### Logistic Regression L1 w/ 10-k CV

In [36]:
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import numpy as np

# Initialize and fit the Logistic Regression model with L1 penalty and cross-validation
log_l1_cv = LogisticRegressionCV(solver='saga', penalty='l1', cv=10, random_state=1)
log_l1_cv.fit(X_train_encoded, y_train)

# Intercept Log-Odds and Odds
print("Intercept (Log-Odds):", log_l1_cv.intercept_)
print("Intercept (Odds):", np.exp(log_l1_cv.intercept_))

# Make predictions on the encoded test data
y_pred = log_l1_cv.predict(X_test_encoded)

# Compute the confusion matrix
cm = confusion_matrix(y_test, y_pred, labels=[0, 1, 2])

# Accuracy
logl1_cv_acc = accuracy_score(y_test, y_pred)

# Precision, Recall (Sensitivity), and F1-Score for each class
logl1_cv_prec = precision_score(y_test, y_pred, average=None, labels=[0, 1, 2])
logl1_cv_rec = recall_score(y_test, y_pred, average=None, labels=[0, 1, 2])
logl1_cv_f1 = f1_score(y_test, y_pred, average=None, labels=[0, 1, 2])

# Specificity calculation for each class
specificity = np.array([
    (np.sum(cm) - np.sum(cm[:, i]) - np.sum(cm[i, :]) + cm[i, i]) / (np.sum(cm) - np.sum(cm[:, i]))
    for i in range(cm.shape[0])
])

# Extract metrics for class 2 (index 2)
class_index = 2
logl1_cv_prec_class2 = logl1_cv_prec[class_index]
logl1_cv_rec_class2 = logl1_cv_rec[class_index]
logl1_cv_f1_class2 = logl1_cv_f1[class_index]
logl1_cv_spec_class2 = specificity[class_index]

# Print metrics
print(f"Accuracy: {logl1_cv_acc:.3f}")
print(f"Class 2 Precision: {logl1_cv_prec_class2:.3f}")
print(f"Class 2 Specificity: {logl1_cv_spec_class2:.3f}")
print(f"Class 2 Recall: {logl1_cv_rec_class2:.3f}")
print(f"Class 2 F1-Score: {logl1_cv_f1_class2:.3f}")


Intercept (Log-Odds): [-1.14510983e-04  1.14684618e-04 -1.73635708e-07]
Intercept (Odds): [0.9998855  1.00011469 0.99999983]
Accuracy: 0.812
Class 2 Precision: 0.000
Class 2 Specificity: 0.842
Class 2 Recall: 0.000
Class 2 F1-Score: 0.000


In [34]:
# Create table of coefficient odds
d = {'Feature': pd.Series(X_train_encoded.columns), 'LogOdds': pd.Series(log_l1_cv.coef_[0])}
df = pd.DataFrame(data=d).reindex(d['LogOdds'].abs().sort_values(ascending=False).index)
df['Odds'] = np.exp(df['LogOdds'])
df.head()

,Feature,LogOdds,Odds
1,year,-0.000782,0.999218
0,weight,0.000000,1.000000
2,age,0.000000,1.000000
3,sex_2,0.000000,1.000000
4,expedited_2,0.000000,1.000000


In [100]:
# Cross val performance

### Logistic Regression Elastic Net

In [38]:
param_grid = {
    'l1_ratio': [i / 9 for i in range(10)],  # 10 values from 0 to 1 (0, 0.1, 0.2, ..., 1.0)
    'C': [0.01, 0.1, 1, 10, 100]  # Different strengths of regularization
}

# Initialize the Logistic Regression model with Elastic Net penalty
log_reg_elasticnet = LogisticRegression(
    penalty='elasticnet',  # Use Elastic Net regularization
    solver='saga',        # Solver that supports Elastic Net
    random_state=1
)

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=log_reg_elasticnet,
    param_grid=param_grid,
    scoring='accuracy',   # Or another metric of choice
    cv=5,                 # Number of cross-validation folds
    n_jobs=-1              # Use all available CPUs
)

# Fit GridSearchCV to the training data
grid_search.fit(X_train_encoded, y_train.values.ravel())

# Extract and print the best parameters and best score from grid search
best_params = grid_search.best_params_
print(f"Best parameters: {best_params}")

best_score = grid_search.best_score_
print(f"Best cross-validation score: {best_score:.3f}")

# Get the best model
best_model = grid_search.best_estimator_

# Make predictions with the best model on the encoded test data
y_pred = best_model.predict(X_test_encoded)

# Compute the confusion matrix
cm = confusion_matrix(y_test, y_pred, labels=[0, 1, 2])

# Accuracy
best_model_acc = accuracy_score(y_test, y_pred)

# Precision, Recall (Sensitivity), and F1-Score for each class
best_model_prec = precision_score(y_test, y_pred, average=None, labels=[0, 1, 2])
best_model_rec = recall_score(y_test, y_pred, average=None, labels=[0, 1, 2])
best_model_f1 = f1_score(y_test, y_pred, average=None, labels=[0, 1, 2])

# Specificity calculation for each class
specificity = np.array([
    (np.sum(cm) - np.sum(cm[:, i]) - np.sum(cm[i, :]) + cm[i, i]) / (np.sum(cm) - np.sum(cm[:, i]))
    for i in range(cm.shape[0])
])

# Extract metrics for class 2 (index 2)
class_index = 2
best_model_prec_class2 = round(best_model_prec[class_index], 3)
best_model_rec_class2 = round(best_model_rec[class_index], 3)
best_model_f1_class2 = round(best_model_f1[class_index], 3)
best_model_spec_class2 = round(specificity[class_index], 3)
best_model_acc = round(best_model_acc, 3)

# Print metrics for the best model
print(f"Accuracy: {best_model_acc:.3f}")
print(f"Class 2 Precision: {best_model_prec_class2:.3f}")
print(f"Class 2 Specificity: {best_model_spec_class2:.3f}")
print(f"Class 2 Recall: {best_model_rec_class2:.3f}")
print(f"Class 2 F1-Score: {best_model_f1_class2:.3f}")


Best parameters: {'C': 0.01, 'l1_ratio': 0.0}
Best cross-validation score: 0.790
Accuracy: 0.812
Class 2 Precision: 0.000
Class 2 Specificity: 0.842
Class 2 Recall: 0.000
Class 2 F1-Score: 0.000


### SINGLE DECISION TREE

In [40]:
# Define a grid of hyperparameters to search for in tree
param_grid = {
    'max_depth' : [1,2,3,4,5],
    'min_samples_leaf' : [1,2,3,4,5]
    
}
# Create a GridSearchCV object using and the defined parameter grid
tree1_search = GridSearchCV(DecisionTreeClassifier(random_state=1), param_grid, cv=10, n_jobs=-1)
# Fit the GridSearchCV to the balanced training data to find the best hyperparameters
tree1_search.fit(X_train_encoded, y_train.values.ravel())
# Get the best hyperparameters found by the GridSearch
tree1_search.best_params_

{'max_depth': 5, 'min_samples_leaf': 1}

In [41]:
tree1 = DecisionTreeClassifier(max_depth = 5, min_samples_leaf = 1, random_state = 1).fit(X_train_encoded, y_train)

In [42]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import numpy as np

# Initialize and fit the Decision Tree model
tree1 = DecisionTreeClassifier(max_depth=5, min_samples_leaf=1, random_state=1)
tree1.fit(X_train_encoded, y_train)

# Make predictions on the encoded test data
y_pred_tree = tree1.predict(X_test_encoded)

# Compute the confusion matrix
cm_tree = confusion_matrix(y_test, y_pred_tree, labels=[0, 1, 2])

# Accuracy
tree1_acc = accuracy_score(y_test, y_pred_tree)

# Precision, Recall (Sensitivity), and F1-Score for each class
tree1_prec = precision_score(y_test, y_pred_tree, average=None, labels=[0, 1, 2])
tree1_rec = recall_score(y_test, y_pred_tree, average=None, labels=[0, 1, 2])
tree1_f1 = f1_score(y_test, y_pred_tree, average=None, labels=[0, 1, 2])

# Specificity calculation for each class
specificity_tree = np.array([
    (np.sum(cm_tree) - np.sum(cm_tree[:, i]) - np.sum(cm_tree[i, :]) + cm_tree[i, i]) / (np.sum(cm_tree) - np.sum(cm_tree[:, i]))
    for i in range(cm_tree.shape[0])
])

# Extract metrics for class 2 (index 2)
class_index = 2
tree1_prec_class2 = round(tree1_prec[class_index], 3)
tree1_rec_class2 = round(tree1_rec[class_index], 3)
tree1_f1_class2 = round(tree1_f1[class_index], 3)
tree1_spec_class2 = round(specificity_tree[class_index], 3)
tree1_acc = round(tree1_acc, 3)

# Print metrics for the Decision Tree model
print(f"Decision Tree Accuracy: {tree1_acc:.3f}")
print(f"Class 2 Precision: {tree1_prec_class2:.3f}")
print(f"Class 2 Specificity: {tree1_spec_class2:.3f}")
print(f"Class 2 Recall: {tree1_rec_class2:.3f}")
print(f"Class 2 F1-Score: {tree1_f1_class2:.3f}")


Decision Tree Accuracy: 0.888
Class 2 Precision: 0.833
Class 2 Specificity: 0.904
Class 2 Recall: 0.443
Class 2 F1-Score: 0.579


In [45]:
# Get feature importance scores
importances=tree1.feature_importances_
feature_importance_pairs=list(zip(X_train_encoded.columns, importances))

# Sort in descending order
sorted_feature_importance_pairs = sorted(feature_importance_pairs, key=lambda x: x[1], reverse=True)

# Print scores
print("Feature Importance Scores")
for feature, importance in sorted_feature_importance_pairs:
    print(f"{feature}: {importance:.4f}")

Feature Importance Scores
age: 0.5002
weight: 0.3622
expedited_2: 0.1108
sex_2: 0.0268
year: 0.0000


/Users/vicky/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/vicky/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/vicky/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/vicky/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/vicky/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/vicky/opt/anaconda3/lib/python3.8/site-pack

## Ensembles

### Random Forest Classifier w/ 10-k CV

In [66]:
#Random Forest
# Create a Random Forest classifier with 100 trees
random_forest = RandomForestClassifier(random_state=1)  

# Fit (train) the Random Forest classifier on the balanced training data
random_forest.fit(X_train_encoded, y_train.values.ravel())

RandomForestClassifier(n_estimators=1000, random_state=1)

In [81]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier

# Define the number of estimators to test
n_estimators_list = [10, 20, 30, 40, 50, 60]

# Initialize lists to store the results
results = []

# Loop over the number of estimators
for n_estimators in n_estimators_list:
    # Initialize and fit the RandomForest model
    random_forest = RandomForestClassifier(random_state=1, n_estimators=n_estimators)
    random_forest.fit(X_train_encoded, y_train.values.ravel())

    # Predict on the test data
    y_pred = random_forest.predict(X_test_encoded)

    # Ensure y_test and y_pred are numpy arrays for consistency
    y_test = np.array(y_test)
    y_pred = np.array(y_pred)

    # Check if dimensions match
    assert y_test.shape[0] == y_pred.shape[0], "Mismatch in dimensions of y_test and y_pred"

    # Compute overall accuracy
    overall_accuracy = accuracy_score(y_test, y_pred)

    # Compute metrics for Class 2
    class_2_precision = precision_score(y_test, y_pred, labels=[2], average='macro', zero_division=0)
    class_2_recall = recall_score(y_test, y_pred, labels=[2], average='macro', zero_division=0)
    class_2_f1 = f1_score(y_test, y_pred, labels=[2], average='macro', zero_division=0)

    # Compute confusion matrix
    cm = confusion_matrix(y_test, y_pred, labels=[0, 1, 2])
    TN = cm[0, 0] + cm[1, 1]  # True Negatives for Class 2
    FP = cm[0, 2] + cm[1, 2]  # False Positives for Class 2

    # Specificity for Class 2
    specificity_class_2 = TN / (TN + FP) if (TN + FP) > 0 else 0

    # Store the results
    results.append({
        'n_estimators': n_estimators,
        'Overall Accuracy': overall_accuracy,
        'Precision for Class 2': class_2_precision,
        'Recall for Class 2': class_2_recall,
        'F1-Score for Class 2': class_2_f1,
        'Specificity for Class 2': specificity_class_2
    })

# Print the results for all tested estimators
for result in results:
    print(f"\nRandom Forest with {result['n_estimators']} estimators:")
    print(f"Overall Accuracy: {result['Overall Accuracy']:.4f}")
    print(f"Precision for Class 2: {result['Precision for Class 2']:.4f}")
    print(f"Recall for Class 2: {result['Recall for Class 2']:.4f}")
    print(f"F1-Score for Class 2: {result['F1-Score for Class 2']:.4f}")
    print(f"Specificity for Class 2: {result['Specificity for Class 2']:.4f}")



Random Forest with 10 estimators:
Overall Accuracy: 0.9720
Precision for Class 2: 0.9048
Recall for Class 2: 0.9620
F1-Score for Class 2: 0.9325
Specificity for Class 2: 0.9809

Random Forest with 20 estimators:
Overall Accuracy: 0.9720
Precision for Class 2: 0.9048
Recall for Class 2: 0.9620
F1-Score for Class 2: 0.9325
Specificity for Class 2: 0.9809

Random Forest with 30 estimators:
Overall Accuracy: 0.9740
Precision for Class 2: 0.9268
Recall for Class 2: 0.9620
F1-Score for Class 2: 0.9441
Specificity for Class 2: 0.9856

Random Forest with 40 estimators:
Overall Accuracy: 0.9740
Precision for Class 2: 0.9268
Recall for Class 2: 0.9620
F1-Score for Class 2: 0.9441
Specificity for Class 2: 0.9856

Random Forest with 50 estimators:
Overall Accuracy: 0.9740
Precision for Class 2: 0.9157
Recall for Class 2: 0.9620
F1-Score for Class 2: 0.9383
Specificity for Class 2: 0.9833

Random Forest with 60 estimators:
Overall Accuracy: 0.9780
Precision for Class 2: 0.9268
Recall for Class 2: 

In [86]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier

# Define the number of estimators to test
n_estimators_list = [10, 20, 30, 40, 50, 60]

# Initialize lists to store the results
results = []

# Loop over the number of estimators
for n_estimators in n_estimators_list:
    # Initialize and fit the RandomForest model
    random_forest = RandomForestClassifier(random_state=1, n_estimators=n_estimators)
    random_forest.fit(X_train_encoded, y_train.values.ravel())

    # Predict on the test data
    y_pred = random_forest.predict(X_test_encoded)

    # Ensure y_test and y_pred are numpy arrays for consistency
    y_test = np.array(y_test)
    y_pred = np.array(y_pred)

    # Check if dimensions match
    assert y_test.shape[0] == y_pred.shape[0], "Mismatch in dimensions of y_test and y_pred"

    # Compute overall accuracy
    overall_accuracy = accuracy_score(y_test, y_pred)

    # Compute metrics for Class 2
    class_2_precision = precision_score(y_test, y_pred, labels=[2], average='macro', zero_division=0)
    class_2_recall = recall_score(y_test, y_pred, labels=[2], average='macro', zero_division=0)
    class_2_f1 = f1_score(y_test, y_pred, labels=[2], average='macro', zero_division=0)

    # Compute confusion matrix
    cm = confusion_matrix(y_test, y_pred, labels=[0, 1, 2])
    TN = cm[0, 0] + cm[1, 1]  # True Negatives for Class 2
    FP = cm[0, 2] + cm[1, 2]  # False Positives for Class 2

    # Specificity for Class 2
    specificity_class_2 = TN / (TN + FP) if (TN + FP) > 0 else 0

    # Store the results
    results.append({
        'n_estimators': n_estimators,
        'Confusion Matrix': cm,
        'Overall Accuracy': overall_accuracy,
        'Precision for Class 2': class_2_precision,
        'Recall for Class 2': class_2_recall,
        'F1-Score for Class 2': class_2_f1,
        'Specificity for Class 2': specificity_class_2
    })

# Print the results for all tested estimators
for result in results:
    print(f"\nRandom Forest with {result['n_estimators']} estimators:")
    print(f"Confusion Matrix:\n{result['Confusion Matrix']}")
    print(f"Overall Accuracy: {result['Overall Accuracy']:.4f}")
    print(f"Precision for Class 2: {result['Precision for Class 2']:.4f}")
    print(f"Recall for Class 2: {result['Recall for Class 2']:.4f}")
    print(f"F1-Score for Class 2: {result['F1-Score for Class 2']:.4f}")
    print(f"Specificity for Class 2: {result['Specificity for Class 2']:.4f}")

# Specifically for 60 estimators
best_n_estimators = 60
best_model = RandomForestClassifier(random_state=1, n_estimators=best_n_estimators)
best_model.fit(X_train_encoded, y_train.values.ravel())
y_pred_best = best_model.predict(X_test_encoded)

cm_best = confusion_matrix(y_test, y_pred_best, labels=[0, 1, 2])
TN_best = cm_best[0, 0] + cm_best[1, 1]  # True Negatives for Class 2
FP_best = cm_best[0, 2] + cm_best[1, 2]  # False Positives for Class 2

specificity_class_2_best = TN_best / (TN_best + FP_best) if (TN_best + FP_best) > 0 else 0

print(f"\nFor Random Forest with {best_n_estimators} estimators:")
print(f"Confusion Matrix:\n{cm_best}")
print(f"Overall Accuracy: {accuracy_score(y_test, y_pred_best):.4f}")
print(f"Precision for Class 2: {precision_score(y_test, y_pred_best, labels=[2], average='macro', zero_division=0):.4f}")
print(f"Recall for Class 2: {recall_score(y_test, y_pred_best, labels=[2], average='macro', zero_division=0):.4f}")
print(f"F1-Score for Class 2: {f1_score(y_test, y_pred_best, labels=[2], average='macro', zero_division=0):.4f}")
print(f"Specificity for Class 2: {specificity_class_2_best:.4f}")



Random Forest with 10 estimators:
Confusion Matrix:
[[ 13   2   0]
 [  1 397   8]
 [  0   3  76]]
Overall Accuracy: 0.9720
Precision for Class 2: 0.9048
Recall for Class 2: 0.9620
F1-Score for Class 2: 0.9325
Specificity for Class 2: 0.9809

Random Forest with 20 estimators:
Confusion Matrix:
[[ 13   2   0]
 [  1 397   8]
 [  0   3  76]]
Overall Accuracy: 0.9720
Precision for Class 2: 0.9048
Recall for Class 2: 0.9620
F1-Score for Class 2: 0.9325
Specificity for Class 2: 0.9809

Random Forest with 30 estimators:
Confusion Matrix:
[[ 13   2   0]
 [  2 398   6]
 [  0   3  76]]
Overall Accuracy: 0.9740
Precision for Class 2: 0.9268
Recall for Class 2: 0.9620
F1-Score for Class 2: 0.9441
Specificity for Class 2: 0.9856

Random Forest with 40 estimators:
Confusion Matrix:
[[ 13   2   0]
 [  2 398   6]
 [  0   3  76]]
Overall Accuracy: 0.9740
Precision for Class 2: 0.9268
Recall for Class 2: 0.9620
F1-Score for Class 2: 0.9441
Specificity for Class 2: 0.9856

Random Forest with 50 estimator

In [82]:
# Get feature importance scores
importances=random_forest.feature_importances_
feature_importance_pairs=list(zip(X_train_encoded.columns, importances))

# Sort in descending order
sorted_feature_importance_pairs = sorted(feature_importance_pairs, key=lambda x: x[1], reverse=True)

# Print scores
print("Feature Importance Scores")
for feature, importance in sorted_feature_importance_pairs:
    print(f"{feature}: {importance:.4f}")

Feature Importance Scores
age: 0.4641
weight: 0.4436
expedited_2: 0.0629
sex_2: 0.0294
year: 0.0000


In [83]:
#generatge plot

In [70]:
# Extract feature names and importances
feature_names, importances = zip(*sorted_feature_importance_pairs)

# Create a horizontal bar plot
plt.figure(figsize=(14, 12))
#plt.barh(range(len(feature_names)), importances, align='center')
#plt.yticks(range(len(feature_names)), feature_names)

plt.barh(range(len(feature_names)), importances, align='center')
plt.yticks(range(len(feature_names)), feature_names)

plt.xlabel('Feature Importance')
plt.gca().invert_yaxis()  # Invert the y-axis to show the most important features at the top
#plt.show()
plt.savefig('featimp.png')
plt.show()

In [ ]:
# Cross val peformance

## Gradient Boosted Tree w/ 10-k CV

In [56]:
# Define a grid of hyperparameters to search for in the Gradient Boosting Classifier
param_grid = {
    'max_depth' : [1,2,3,4,5],
    'learning_rate' : [0.0001, 0.001, 0.01, 0.1, 1],
    'n_estimators' : [20,21,22,23,24],
}
# Create a GridSearchCV object using GradientBoostingClassifier and the defined parameter grid
tree_search = GridSearchCV(GradientBoostingClassifier(random_state=1), param_grid, cv=10, n_jobs=-1)
# Fit the GridSearchCV to the balanced training data to find the best hyperparameters
tree_search.fit(X_train_encoded, y_train.values.ravel())
# Get the best hyperparameters found by the GridSearch
tree_search.best_params_

{'learning_rate': 1, 'max_depth': 5, 'n_estimators': 20}

In [64]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

# Assuming X_test_encoded and y_test are defined and best_model is obtained from GridSearchCV

# Get the best model from GridSearchCV
best_model = tree_search.best_estimator_

# Predict on the test data
y_pred = best_model.predict(X_test_encoded)

# Ensure y_test and y_pred are numpy arrays for consistency
y_test = np.array(y_test)
y_pred = np.array(y_pred)

# Check if dimensions match
assert y_test.shape[0] == y_pred.shape[0], "Mismatch in dimensions of y_test and y_pred"

# Compute overall accuracy
overall_accuracy = accuracy_score(y_test, y_pred)

# Compute metrics for Class 2
class_2_precision = precision_score(y_test, y_pred, labels=[2], average='macro', zero_division=0)
class_2_recall = recall_score(y_test, y_pred, labels=[2], average='macro', zero_division=0)
class_2_f1 = f1_score(y_test, y_pred, labels=[2], average='macro', zero_division=0)

# Compute confusion matrix
cm = confusion_matrix(y_test, y_pred, labels=[0, 1, 2])
TN = cm[1, 1] + cm[0, 0]  # True Negatives for Class 2
FP = cm[0, 1] + cm[1, 0]  # False Positives for Class 2

# Specificity for Class 2
specificity_class_2 = TN / (TN + FP) if (TN + FP) > 0 else 0

# Print the results
print(f'Best Hyperparameters: {tree_search.best_params_}')
print(f'Overall Accuracy: {overall_accuracy:.4f}')
print(f'Precision for Class 2: {class_2_precision:.4f}')
print(f'Recall for Class 2: {class_2_recall:.4f}')
print(f'F1-Score for Class 2: {class_2_f1:.4f}')
print(f'Specificity for Class 2: {specificity_class_2:.4f}')


Best Hyperparameters: {'learning_rate': 1, 'max_depth': 5, 'n_estimators': 20}
Overall Accuracy: 0.8160
Precision for Class 2: 0.5152
Recall for Class 2: 0.6456
F1-Score for Class 2: 0.5730
Specificity for Class 2: 0.9571


## Non-parametric

### K-Nearest Neighbors Classifier

In [75]:
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt

# Initialize variables to track the best model and performance
best_k = None
best_accuracy = 0
best_model = None
best_y_pred = None

# Test k-values from 1 to 5
for k in range(1, 6):
    # Initialize and train the KNN model
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train_encoded, y_train)
    
    # Predict on the test data
    y_pred = knn.predict(X_test_encoded)
    
    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)
    
    # Update the best model if this one has a higher accuracy
    if accuracy > best_accuracy:
        best_k = k
        best_accuracy = accuracy
        best_model = knn
        best_y_pred = y_pred

# Output the best k and its accuracy
print(f"Best k-value: {best_k}")
print(f"Best accuracy: {best_accuracy:.3f}")

# Compute the confusion matrix
cm = confusion_matrix(y_test, best_y_pred, labels=[0, 1, 2])

# Create a DataFrame for better visualization
cm_df = pd.DataFrame(cm, index=[f'Class {i}' for i in range(cm.shape[0])],
                     columns=[f'Predicted Class {i}' for i in range(cm.shape[1])])

# Print the confusion matrix
print("\nConfusion Matrix:")
print(cm_df)

# Plot the confusion matrix
plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=[f'Class {i}' for i in range(cm.shape[1])], 
            yticklabels=[f'Class {i}' for i in range(cm.shape[0])])
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')

# Show the plot
plt.tight_layout()  # Adjusts plot to fit labels and titles
plt.show()

# Compute and print detailed classification report
report = classification_report(y_test, best_y_pred, labels=[0, 1, 2], target_names=[f'Class {i}' for i in range(3)])
print("\nClassification Report:")
print(report)


Best k-value: 1
Best accuracy: 0.956

Confusion Matrix:
         Predicted Class 0  Predicted Class 1  Predicted Class 2
Class 0                  9                  5                  1
Class 1                  3                393                 10
Class 2                  0                  3                 76

Classification Report:
              precision    recall  f1-score   support

     Class 0       0.75      0.60      0.67        15
     Class 1       0.98      0.97      0.97       406
     Class 2       0.87      0.96      0.92        79

    accuracy                           0.96       500
   macro avg       0.87      0.84      0.85       500
weighted avg       0.96      0.96      0.96       500



In [76]:
#manually make heatmap 

data = {
    'Predicted Class 0': [9, 3, 0],
    'Predicted Class 1': [5, 393, 3],
    'Predicted Class 2': [1, 10, 76]
}

# Create a DataFrame for the confusion matrix
cm_df = pd.DataFrame(data, index=['Class 0', 'Class 1', 'Class 2'])

# Plot the confusion matrix as a heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(cm_df, annot=True, fmt='d', cmap='Blues', 
            xticklabels=cm_df.columns, yticklabels=cm_df.index)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix Heatmap')
plt.tight_layout()  # Adjusts plot to fit labels and titles
plt.show()

In [ ]:
# Create a K-Nearest Neighbors (KNN) classifier with 6 neighbors and fit it to the balanced training data
knn = KNeighborsClassifier(n_neighbors = 5).fit(X_train_under, y_train_under.values.ravel())

## Neural Network

In [ ]:
# train

In [ ]:
# feature importance

In [ ]:
# 

# Validation Data Tuning

# Testing Performance

## Logistic Regression

### Predictions

In [149]:
X_train_pipe

,med_product_filgrastim,med_product_iressa,med_product_lenvatinib,med_product_xolair,med_product_xyrem,sex_2,expedited_2,weight,year
0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,-1.159326,0.0
1,0.0,1.0,0.0,0.0,0.0,1.0,0.0,-1.159326,0.0
2,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.563567,0.0
3,0.0,1.0,0.0,0.0,0.0,1.0,0.0,-1.159326,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,1.0,-0.383712,0.0
5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.019654,0.0
6,0.0,1.0,0.0,0.0,0.0,1.0,0.0,-1.159326,0.0
7,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.019654,0.0
8,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.019654,0.0
9,0.0,0.0,1.0,0.0,0.0,0.0,1.0,-0.383712,0.0


ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- sex_1
Feature names seen at fit time, yet now missing:
- med_product_filgrastim
- med_product_iressa
- med_product_lenvatinib
- med_product_xyrem
- sex_2


### Confusion Matrix

In [ ]:
#Logistic Regression
log_cm = confusion_matrix(y_true = y_test, y_pred = log_l1.predict(X_test_log), labels = log_l2_bal.classes_)
log_disp = ConfusionMatrixDisplay(confusion_matrix= log_cm_bal, display_labels=log_l2_bal.classes_)
log_disp.plot()
#plt.show()
plt.savefig('lr_cm.png')

In [ ]:
# classification summary

In [ ]:
# curve

In [ ]:
#Sensitivity, specificity, and accuracy for log reg
logl1_sens = sensitivity_score(y_test, log_l1.predict(X_test))
logl1_spec = specificity_score(y_test, log_l1.predict(X_test))
logl1_acc = accuracy_score(y_test, log_l1.predict(X_test))
logl1_prec = precision_score(y_test, log_l1.predict(X_test)) 
logl1_rec = recall_score(y_test, log_l1.predict(X_test))
logl1_f1 = f1_score(y_test, log_l1.predict(X_test))
logl1_sens, logl1_spec, logl1_acc, logl1_prec, logl1_rec, logl1_f1

## Decision Tree

### Predictions

### Confusion Matrix

In [ ]:
#Confusion matrix for tree
tree_cm = confusion_matrix(y_true = y_test, y_pred = tree1.predict(X_test), labels = tree1.classes_)
tree_disp = ConfusionMatrixDisplay(confusion_matrix= tree_cm, display_labels=tree1.classes_)
tree_disp.plot()
plt.show() 

In [ ]:
# classification summary

In [ ]:
# curve

In [ ]:
#Sensitivity, specificity, and accuracy for decision tree
tree1_sens = sensitivity_score(y_test, tree1.predict(X_test))
tree1_spec = specificity_score(y_test, tree1.predict(X_test))
tree1_acc = accuracy_score(y_test, tree1.predict(X_test))
tree1_prec = precision_score(y_test, tree1.predict(X_test)) 
tree1_rec = recall_score(y_test, tree1.predict(X_test))
tree1_f1 = f1_score(y_test, tree1.predict(X_test))
tree1_sens, tree1_spec, tree1_acc, tree1_prec, tree1_rec, tree1_f1

## Random Forest

### Predictions

### Confusion Matrix

In [ ]:
# Compute the confusion matrix for RF 
rf_cm = confusion_matrix(y_true = y_test, y_pred = random_forest.predict(X_test), labels = random_forest.classes_)
# Create a ConfusionMatrixDisplay object for visualization
rf_disp = ConfusionMatrixDisplay(confusion_matrix= rf_cm, display_labels=random_forest.classes_)
rf_disp.plot()
warnings.filterwarnings('ignore')
#plt.show()
#plt.savefig('rf_cm.png')

In [1]:
# classification summary

In [ ]:
# curve

In [ ]:
#Sensitivity, specificity, and accuracy for RF
rf_sens = sensitivity_score(y_test, random_forest.predict(X_test))
rf_spec = specificity_score(y_test, random_forest.predict(X_test))
rf_acc = accuracy_score(y_test, random_forest.predict(X_test))
rf_prec = precision_score(y_test, random_forest.predict(X_test)) 
rf_rec = recall_score(y_test, random_forest.predict(X_test))
rf_f1 = f1_score(y_test, random_forest.predict(X_test))
warnings.filterwarnings('ignore')
rf_sens, rf_spec, rf_acc, rf_prec, rf_rec, rf_f1

## Gradient Boosted Tree

### Predictions

### Confusion Matrix

In [ ]:
#Confusion matrix for tree
tree_cm = confusion_matrix(y_true = y_test, y_pred = tree_search.predict(X_test), labels = tree_search.classes_)
tree_disp = ConfusionMatrixDisplay(confusion_matrix= tree_cm, display_labels=tree_search.classes_)
tree_disp.plot()
plt.show() 

In [2]:
# classification summary

In [ ]:
# curve

In [ ]:
#Sensitivity, specificity, and accuracy for decision tree 
tree_sens = sensitivity_score(y_test, tree_search.predict(X_test))
tree_spec = specificity_score(y_test, tree_search.predict(X_test))
tree_acc = accuracy_score(y_test, tree_search.predict(X_test))
tree_prec = precision_score(y_test, tree_search.predict(X_test)) 
tree_rec = recall_score(y_test, tree_search.predict(X_test))
tree_f1 = f1_score(y_test, tree_search.predict(X_test))
tree_sens, tree_spec, tree_acc, tree_prec, tree_rec, tree_f1

## K-Nearest Neighbors

### Predictions

### Confusion Matrix

In [ ]:
#KNN
# Compute the confusion matrix for KNN 
knn_cm = confusion_matrix(y_true = y_test, y_pred = knn.predict(X_test), labels = knn.classes_)
# Create a ConfusionMatrixDisplay object for visualization
knn_disp = ConfusionMatrixDisplay(confusion_matrix= knn_cm, display_labels=knn.classes_)
knn_disp.plot()
warnings.filterwarnings('ignore')
plt.show()

In [ ]:
# Classification summary

In [ ]:
# Curve

In [ ]:
#Sensitivity, specificity, and accuracy for KNN 
knn_sens = sensitivity_score(y_test, knn.predict(X_test))
knn_spec = specificity_score(y_test, knn.predict(X_test))
knn_acc = accuracy_score(y_test, knn.predict(X_test))
knn_prec = precision_score(y_test, knn.predict(X_test)) 
knn_rec = recall_score(y_test, knn.predict(X_test))
knn_f1 = f1_score(y_test, knn.predict(X_test))
warnings.filterwarnings('ignore')
knn_sens, knn_spec, knn_acc, knn_prec, knn_rec, knn_f1

## Neural Network

### Predictions

### Confusion Matrix

In [ ]:
# Classification summary

In [ ]:
# Curve

# Performance Metrics Table

In [ ]:
# Create a table to display performance metrics for different models on the validation dataset

val_performance = [
{'Model': 'Neural Net', 'Test Sensitivity': nn_sens, 'Test Specificity':nn_spec, 
 'Accuracy': nn_acc, 'Test Precision': nn_prec, 'Test Recall': nn_rec, 'F1 Score': nn_f1},
    {'Model': 'Logistic Regression', 'Test Sensitivity': logl1_sens, 'Test Specificity':logl1_spec, 
 'Accuracy': logl1_acc, 'Test Precision': logl1_prec, 'Test Recall': logl1_rec,  'F1 Score': logl1_f1},
    {'Model': 'Boosted Tree', 'Test Sensitivity': tree_sens, 'Test Specificity':tree_spec, 
 'Accuracy': tree_acc, 'Test Precision': tree_prec, 'Test Recall': tree_rec,  'F1 Score': tree_f1},
    {'Model': 'Single Tree', 'Test Sensitivity': tree1_sens, 'Test Specificity':tree1_spec, 
 'Accuracy': tree1_acc, 'Test Precision': tree1_prec, 'Test Recall': tree1_rec, 'F1 Score': tree1_f1},
    {'Model': 'Random Forest', 'Test Sensitivity': rf_sens, 'Test Specificity':rf_spec, 
 'Accuracy': rf_acc, 'Test Precision': rf_prec, 'Test Recall': rf_rec, 'F1 Score': rf_f1},
    {'Model': 'K-Nearest Neighbors', 'Test Sensitivity': knn_sens, 'Test Specificity':knn_spec, 
 'Accuracy': knn_acc, 'Test Precision': knn_prec, 'Test Recall': knn_rec,  'F1 Score': knn_f1},
    
]
# Create a formatted table using tabulate and specify the format as 'fancy_grid'
table = tabulate(val_performance, headers='keys', tablefmt='fancy_grid')
# Display the comparison table
print(table)